In [40]:
import rsa
import multiprocessing

Reference:

rsa: https://stuvel.eu/python-rsa-doc/usage.html#signing-and-verification  

cryptography: https://github.com/pyca/cryptography

# environment

## install rsa

`pip install rsa`

The RSA in pycryptodome is difficult to use

## install cryptography

`pip install cryptography`

# Generating keys

In [41]:
(pubkey, privkey) = rsa.newkeys(512)
print (pubkey)
print (privkey)

PublicKey(12861544132625847745033511106601970583073708486957997771962678921008411920684869745860964221202086185044727195672199860866945731999794236297103215122745791, 65537)
PrivateKey(12861544132625847745033511106601970583073708486957997771962678921008411920684869745860964221202086185044727195672199860866945731999794236297103215122745791, 65537, 6817283673026789431420023937945576607333483153332421786203816780719139006112952175112295146319471186585606676451641141172512563877850718384367725578103473, 7401878590334007487006911807144262992451520134472500637023913830426735037930016541, 1737605389721134906564725106133593380921966659289810323230138237898234251)


Another way to speed up the key generation process is to use multiple processes in parallel to speed up the key generation. Use no more than the number of processes that your machine can run in parallel; a dual-core machine should use poolsize=2; a quad-core hyperthreading machine can run two threads on each core, and thus can use poolsize=8.

In [42]:
(pubkey, privkey) = rsa.newkeys(512, poolsize=multiprocessing.cpu_count())
print (pubkey)
print (privkey)

PublicKey(7323670490096457572124703713659635429642042745109801914985999819256090280931841321177218408504292931781949766770426136673105546389718672287366568709771553, 65537)
PrivateKey(7323670490096457572124703713659635429642042745109801914985999819256090280931841321177218408504292931781949766770426136673105546389718672287366568709771553, 65537, 6915564464037402945678582927503952554486745125366130453162466985285611894391439677482188826220041142385867064502875062273878424645883918409562411364588613, 5053489826746648397239145139658123327463819229923167059573600032652472468739893979, 1449230282672066490539556102289156426726618291033339381966517405236080307)


# Encryption and decryption

To encrypt or decrypt a message, use rsa.encrypt() resp. rsa.decrypt().
Let’s say that Alice wants to send a message that only Bob can read.

1.Bob generates a keypair, and gives the public key to Alice. This is done such that Alice knows for sure that the key is really Bob’s (for example by handing over a USB stick that contains the key).

In [43]:
(bob_pub, bob_priv) = rsa.newkeys(512)

2.Alice writes a message, and encodes it in UTF-8. The RSA module only operates on bytes, and not on strings, so this step is necessary.

In [44]:
message = 'hello Bob!'.encode('utf8')

3.Alice encrypts the message using Bob’s public key, and sends the encrypted message.

In [45]:
crypto = rsa.encrypt(message, bob_pub)

4.Bob receives the message, and decrypts it with his private key.

In [46]:
message = rsa.decrypt(crypto, bob_priv)
print(message.decode('utf8'))

hello Bob!


*RSA can only encrypt messages that are smaller than the key. A couple of bytes are lost on random padding, and the rest is available for the message itself. *For example, a 512-bit key can encode a 53-byte message (512 bit = 64 bytes, 11 bytes are used for random padding and other stuff). See Working with big files for information on how to work with larger files.

# Signing and verification

create a detached signature
This hashes the message using SHA-1. Other hash methods are also possible
The RSA module only operates on bytes, and not on strings, so this step is necessary.

In [47]:
(pubkey, privkey) = rsa.newkeys(512)
message = 'Go left at the blue tree'
signature = rsa.sign(message.encode('utf8'), privkey, 'SHA-1')
print (signature)

b'\xb3\x06\x0b\xc1A\xb5\x1bY\rPA\x05m\x9c\xf8riV\xf3\x8e\xfcA\xe8\x81\x18\r\xb9zn\xc3rz\x1eHZ`\x84=\x08A\x85w\x80\x16\xaa\x0c\x05p\x88\xfaO;\xdd\r.\xe7\x14~\xdfuW\xe5\x9d\xd9'


verify the signature

In [48]:
message = 'Go left at the blue tree'
rsa.verify(message.encode('utf8'), signature, pubkey)

True

*Never display the stack trace of a rsa.pkcs1.VerificationError exception. It shows where in the code the exception occurred, and thus leaks information about the key. It’s only a tiny bit of information, but every bit makes cracking the keys easier.*

# ChaCha20

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

key = os.urandom(32)
nonce = os.urandom(16)

algorithm = algorithms.ChaCha20(key, nonce)
cipher = Cipher(algorithm, mode=None, backend=default_backend())
encryptor = cipher.encryptor()

message = 'hello Bob你好!'.encode('utf8')
ct = encryptor.update(message)
decryptor = cipher.decryptor()
print(decryptor.update(ct).decode('utf8'))

hello Bob你好!


# Working with big files

RSA can only encrypt messages that are smaller than the key.The most common way to use RSA with larger files uses a block cypher like AES or DES3 to encrypt the file with a random key, then encrypt the random key with RSA. You would send the encrypted file along with the encrypted key to the recipient. The complete flow is:

## Generate a random key

In [143]:
import os
chacha20_key = os.urandom(32) # must be 32 bytes
nonce = os.urandom(16) # nonce
print (aes_key)

b'\x16\xd7/\xbc\xc64Tg\xb7AP"wX\xa8n\xfeL$\xf6\xce\xa8\xa5\xb5\xef\xe57\xb6\xf9\xc5\xd5\x1d'


## Use that key to encrypt the file with ChaCha20.

message.txt : original file  
encrypted.bin : encrypted file  
decrypted.txt: decrypted file

### Encrypt data with ChaCha20

In [156]:
with open('message.txt','rb') as f:
    data = f.read()
    f.close()
    
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
encryptor = cipher.encryptor()
ct = encryptor.update(data)
with open('encrypted.bin','wb') as f:
    f.write(ct)
    f.close()



## Encrypt the ChaCha20 key with RSA

In [157]:
(public_rsa_key, privkey) = rsa.newkeys(512)
encrypted_chacha20_key = rsa.encrypt(chacha20_key, public_rsa_key)

4.Send the encrypted file together with encrypted_aes_key

5.The recipient now reverses this process to obtain the encrypted file.

In [161]:
key = rsa.decrypt(encrypted_chacha20_key,privkey)
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
decryptor = cipher.decryptor()
with  open("encrypted.bin", "rb") as file_in:
    encrypted_data = file_in.read()
    file_in.close()

decrypted_data = decryptor.update(encrypted_data)
with open('decrypted.txt','wb') as f:
    f.write(decrypted_data)
    f.close()

In [159]:
print(data)

b"The Witcher 3: Wild Hunt[a] is a 2015 action role-playing video game developed and published by CD Projekt. \r\nBased on The Witcher series of fantasy novels by Polish author Andrzej Sapkowski, \r\nit is the sequel to the 2011 video game The Witcher 2: Assassins of Kings and the third installment in The Witcher video game series. \r\nPlayed in an open world with a third-person perspective, players control protagonist Geralt of Rivia. \r\nGeralt, a monster hunter known as a Witcher, is looking for his missing adopted daughter, \r\nwho is on the run from the Wild Hunt: an otherworldly force determined to capture and use her powers. \r\nPlayers battle the game's many dangers with weapons and magic, interact with non-player characters, \r\nand complete main-story and side quests to acquire experience points and gold, \r\nwhich are used to increase Geralt's abilities and purchase equipment. Its central story has several endings, \r\ndetermined by the player's choices at certain points in 

In [160]:
print(decrypted_data)

b"The Witcher 3: Wild Hunt[a] is a 2015 action role-playing video game developed and published by CD Projekt. \r\nBased on The Witcher series of fantasy novels by Polish author Andrzej Sapkowski, \r\nit is the sequel to the 2011 video game The Witcher 2: Assassins of Kings and the third installment in The Witcher video game series. \r\nPlayed in an open world with a third-person perspective, players control protagonist Geralt of Rivia. \r\nGeralt, a monster hunter known as a Witcher, is looking for his missing adopted daughter, \r\nwho is on the run from the Wild Hunt: an otherworldly force determined to capture and use her powers. \r\nPlayers battle the game's many dangers with weapons and magic, interact with non-player characters, \r\nand complete main-story and side quests to acquire experience points and gold, \r\nwhich are used to increase Geralt's abilities and purchase equipment. Its central story has several endings, \r\ndetermined by the player's choices at certain points in 